In [1]:
import spacy
import json
import random
import pandas as pd

In [2]:
data_file = open('./chatbot_data/intents.json').read()
intents = json.loads(data_file)


In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
intent_list = []
documents = []
responses = []

for i, intent in enumerate(intents['intents']):
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag'], i))
        if intent['tag'] not in intent_list:
            intent_list.append(intent['tag'])
    for response in intent['responses']:
        responses.append((i, response))
responses_df = pd.DataFrame(responses, columns=['no', 'response'])
print(f'例句個數:{len(documents)}, intent個數:{len(intent_list)}')
responses_df

例句個數:47, intent個數:9


,no,response
0,0,"Hello, thanks for asking"
1,0,Good to see you again
2,0,"Hi there, how can I help?"
3,1,See you!
4,1,Have a nice day
5,1,Bye! Come back again soon.
6,2,Happy to help!
7,2,Any time!
8,2,My pleasure
9,3,"Sorry, can't understand you"


In [8]:
nlp = spacy.load('en_core_web_md')


In [9]:
from spacy.lang.en.stop_words import STOP_WORDS


In [15]:
def remove_stopwords(text1):
    filtered_sentence =[] 
    doc = nlp(text1)
    for word in doc:
        if word.is_stop == False:
            filtered_sentence.append(word.lemma_)
    return nlp(' '.join(filtered_sentence))

In [17]:
def say_goodbye():
    tag = 1
    response_filter = responses_df[responses_df['no'] == tag][['response']]
    selected_response = response_filter.sample().iloc[0, 0]
    return selected_response


def say_not_understand():
    tag = 3 
    response_filter = responses_df[responses_df['no'] == tag][['response']]
    selected_response = response_filter.sample().iloc[0, 0]
    return selected_response

In [18]:
prob_thread =0.6
while True:
    max_score = 0
    intent_no = -1
    similar_question = ''
    question = input('請輸入:\n')
    if question == '':
        break
    doc1 = remove_stopwords(question)
    for utterance in documents:
        doc2 = remove_stopwords(utterance[0])
        if len(doc1) > 0 and len(doc2) > 0:
            score = doc1.similarity(doc2)
        if score > max_score:
            max_score = score
            intent_no = utterance[2]
            similar_question = utterance[1] +', '+utterance[0]
    if intent_no == -1 or max_score < prob_thread:
        print(say_not_understand())
    else:
        print(f'你問的是：{similar_question}')
        response_filter = responses_df[responses_df['no'] == intent_no][['response']]
        # print(response_filter)
        selected_response = response_filter.sample().iloc[0, 0]
        # print(type(selected_response))
        print(f'回答：{selected_response}')

# say goodbye!        
print(f'回答：{say_goodbye()}')

請輸入:

回答：See you!
